In [1]:
import findspark
findspark.init()
findspark.find()

'/Users/calvinyu/miniforge3/envs/spark/lib/python3.11/site-packages/pyspark'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, split, udf, lit, sum, avg, max, count
import matplotlib.pyplot as plt
import plotly.express as px
import functools
from pyspark.sql import DataFrame
import pandas as pd

In [7]:
import plotly.io as pio
pio.renderers.default = "iframe"

In [6]:
spark = SparkSession.builder.appName('test').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/11 22:32:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
mapping = {10: "October", 11: "November"}
def process_dfs(_spark, _file_names):
    dfs = []
    for file_name in _file_names:
        df = _spark.read.option("header",True).csv(file_name)
        df = df.withColumn("price", df.price.cast('float'))
        dt_col = split(df["event_time"], " ")
        df = df.withColumn("date", dt_col.getItem(0))
        df = df.withColumn("time", dt_col.getItem(1))
        date_col = split(df["date"], "-")
        df = df.withColumn("year", date_col.getItem(0))
        df = df.withColumn("year", df.year.cast('int'))
        df = df.withColumn("month", date_col.getItem(1))
        df = df.withColumn("month", df.month.cast('int'))
        df = df.withColumn("day", date_col.getItem(2))
        df = df.withColumn("day", df.day.cast('int'))

        df = df.withColumn("month", udf(lambda x : mapping[x])("month"))
        df = df.drop("event_time")
        # df = df.drop("date")
        dfs.append(df)

    df_output = functools.reduce(DataFrame.union, dfs)
    return df_output

df = process_dfs(spark, ["data/2019-Nov.csv", "data/2019-Oct.csv"])

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/calvinyu/Documents/test_programs/HPDC/pyspark_test/data/2019-Nov.csv.

In [130]:
df.count()

109950743

In [131]:
df[["brand"]].distinct().count()

4304

In [129]:
df.show(5, truncate=False)

+----------+----------+-------------------+-------------------------+------+------+---------+------------------------------------+--------+----+--------+---+
|event_type|product_id|category_id        |category_code            |brand |price |user_id  |user_session                        |time    |year|month   |day|
+----------+----------+-------------------+-------------------------+------+------+---------+------------------------------------+--------+----+--------+---+
|view      |1003461   |2053013555631882655|electronics.smartphone   |xiaomi|489.07|520088904|4d3b30da-a5e4-49df-b1a8-ba5943f1dd33|00:00:00|2019|November|1  |
|view      |5000088   |2053013566100866035|appliances.sewing_machine|janome|293.65|530496790|8e5f4f83-366c-4f70-860e-ca7417414283|00:00:00|2019|November|1  |
|view      |17302664  |2053013553853497655|null                     |creed |28.31 |561587266|755422e7-9040-477b-9bd2-6a6e8fd97387|00:00:01|2019|November|1  |
|view      |3601530   |2053013563810775923|appliance

In [12]:
nov_df = spark.read.option("header",True).csv("data/2019-Nov.csv")

In [51]:
nov_df = nov_df.withColumn("price", nov_df.price.cast('float'))

In [13]:
oct_df = spark.read.option("header",True).csv("data/2019-Oct.csv")

In [7]:
nov_df.count()

67501980

In [8]:
oct_df.count()

42448765

In [14]:
nov_df.show(5, truncate = False)

+-----------------------+----------+----------+-------------------+-------------------------+------+------+---------+------------------------------------+
|event_time             |event_type|product_id|category_id        |category_code            |brand |price |user_id  |user_session                        |
+-----------------------+----------+----------+-------------------+-------------------------+------+------+---------+------------------------------------+
|2019-11-01 00:00:00 UTC|view      |1003461   |2053013555631882655|electronics.smartphone   |xiaomi|489.07|520088904|4d3b30da-a5e4-49df-b1a8-ba5943f1dd33|
|2019-11-01 00:00:00 UTC|view      |5000088   |2053013566100866035|appliances.sewing_machine|janome|293.65|530496790|8e5f4f83-366c-4f70-860e-ca7417414283|
|2019-11-01 00:00:01 UTC|view      |17302664  |2053013553853497655|null                     |creed |28.31 |561587266|755422e7-9040-477b-9bd2-6a6e8fd97387|
|2019-11-01 00:00:01 UTC|view      |3601530   |2053013563810775923|app

In [15]:
nov_df.columns

['event_time',
 'event_type',
 'product_id',
 'category_id',
 'category_code',
 'brand',
 'price',
 'user_id',
 'user_session']

### Aggregations for Brands

In [16]:
agg_by_brand = df.filter(df.event_type == "purchase") \
      .groupby("brand") \
      .agg(count("*").alias("count"), \
         sum("price").alias("revenue")) \
      .orderBy("count", ascending=False).toPandas()

In [ ]:
print(len(agg_by_brand))

In [18]:
agg_by_brand.head(5)

,brand,count,revenue
0,samsung,372923,1.012774e+08
1,apple,308937,2.387218e+08
2,None,131487,1.956632e+07
3,xiaomi,124908,2.045390e+07
4,huawei,47204,9.664104e+06


In [19]:
agg_by_brand.to_csv("agg_by_brand.csv", index=False)

### Popular Brands

In [7]:
top_categories = ['samsung', 'apple', 'xiaomi', 'huawei', 'cordiant', 'lucente', 'oppo', 'lg', 'sony', 'artel', 'bosch', 'acer', 'lenovo', 'nokian']
df = df.withColumn("adjusted_brand", udf(lambda x : x if x in top_categories else "other")("brand"))

brand_counts = df.filter(df.event_type == "purchase") \
      .groupby("adjusted_brand") \
      .count()

In [123]:
brand_counts = brand_counts.toPandas()

In [124]:
brand_counts

,adjusted_brand,count
0,cordiant,27534
1,apple,308937
2,other,623127
3,oppo,25971
4,sony,17038
5,lg,21606
6,acer,13284
7,artel,15391
8,samsung,372923
9,lucente,26137


In [129]:
brand_counts.to_csv("brand_counts.csv", index=False)

In [112]:
temp_counts = brand_counts.iloc[:30]

In [128]:
fig = px.pie(brand_counts, values='count', names='adjusted_brand', hole=0.3)
fig.show()

### Daily Trends

In [21]:
temp_table = df.createOrReplaceTempView("temp_table")

In [22]:
highest_thru_nov = spark.sql("WITH CTE AS (SELECT month, day, brand, COUNT(brand) as count FROM temp_table WHERE event_type = 'purchase' GROUP BY month, day, brand ORDER BY count DESC), \
                            CTE_2 AS (SELECT month, day, MAX(count) as max_count FROM CTE WHERE month = 'November' GROUP BY month, day ORDER BY day) \
                            SELECT c1.month, c1.day, c1.brand, max_count FROM CTE c1 JOIN CTE_2 c2 ON c1.month = c2.month and c1.day = c2.day and c1.count = c2.max_count ORDER BY c1.day").toPandas()

In [23]:
highest_thru_oct = spark.sql("WITH CTE AS (SELECT month, day, brand, COUNT(brand) as count FROM temp_table WHERE event_type = 'purchase' GROUP BY month, day, brand ORDER BY count DESC), \
                            CTE_2 AS (SELECT month, day, MAX(count) as max_count FROM CTE WHERE month = 'October' GROUP BY month, day ORDER BY day) \
                            SELECT c1.month, c1.day, c1.brand, max_count FROM CTE c1 JOIN CTE_2 c2 ON c1.month = c2.month and c1.day = c2.day and c1.count = c2.max_count ORDER BY c1.day").toPandas()

In [37]:
highest_thru_nov_rev = spark.sql("WITH CTE AS (SELECT month, day, brand, SUM(price) as revenue FROM temp_table WHERE event_type = 'purchase' GROUP BY month, day, brand ORDER BY revenue DESC), \
                            CTE_2 AS (SELECT month, day, MAX(revenue) as max_revenue FROM CTE WHERE month = 'November' GROUP BY month, day ORDER BY day) \
                            SELECT c1.month, c1.day, c1.brand, max_revenue FROM CTE c1 JOIN CTE_2 c2 ON c1.month = c2.month and c1.day = c2.day and c1.revenue = c2.max_revenue ORDER BY c1.day").toPandas()

In [38]:
highest_thru_oct_rev = spark.sql("WITH CTE AS (SELECT month, day, brand, SUM(price) as revenue FROM temp_table WHERE event_type = 'purchase' GROUP BY month, day, brand ORDER BY revenue DESC), \
                            CTE_2 AS (SELECT month, day, MAX(revenue) as max_revenue FROM CTE WHERE month = 'October' GROUP BY month, day ORDER BY day) \
                            SELECT c1.month, c1.day, c1.brand, max_revenue FROM CTE c1 JOIN CTE_2 c2 ON c1.month = c2.month and c1.day = c2.day and c1.revenue = c2.max_revenue ORDER BY c1.day").toPandas()

In [105]:
merged = pd.merge(highest_thru_oct, highest_thru_nov, how='outer', on='day')
merged = pd.merge(merged, highest_thru_nov_rev, how='outer', on='day')[["day", "brand_x", "brand_y", "max_count_x", "max_count_y", "max_revenue"]].rename({"max_revenue":"max_revenue_x"})
merged = pd.merge(merged, highest_thru_oct_rev, how='outer', on='day').rename({"brand_x":"oct_brand", "brand_y":"nov_brand", "max_count_x":"oct_count", "max_count_y":"nov_count","max_revenue_x":"oct_rev", "max_revenue_y":"nov_rev"}, axis=1)

In [107]:
merged.to_csv("day_stats.csv", index=False)

In [106]:
merged.head(5)

,day,oct_brand,nov_brand,oct_count,nov_count,oct_rev,month,brand,nov_rev
0,1,samsung,samsung,4427,5291.0,3.454626e+06,October,apple,3.175375e+06
1,2,samsung,samsung,4618,5179.0,2.979444e+06,October,apple,3.092671e+06
2,3,samsung,samsung,4590,5332.0,3.199490e+06,October,apple,3.081512e+06
3,4,samsung,apple,6368,5548.0,4.186447e+06,October,apple,4.165188e+06
4,5,samsung,samsung,5323,5125.0,3.643563e+06,October,apple,3.488695e+06


In [103]:
fig = px.bar(merged, y="day", x=["oct_count", "nov_count"], orientation='h')
fig.update_layout(width=750, height=750,
legend=dict(
        x=0.8,
        y=0.8,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
    ))

In [104]:
fig = px.bar(merged, y="day", x=["oct_rev", "nov_rev"], orientation='h')
fig.update_layout(xaxis=dict(autorange="reversed"), width=750, height=750,
legend=dict(
        x=0.1,
        y=0.8,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
    ))
fig.update_yaxes(visible=False)


### User Event Trends

In [8]:
temp_table = df.filter(df["brand"].isin(top_categories[:5])).limit(3000000).createOrReplaceTempView("small_table")

In [8]:
view_to_purchase = spark.sql(
    "WITH CTE AS (SELECT \
    v.brand, \
    v.product_id,\
    v.user_session,\
    v.time AS view_time,\
    p.time AS purchase_time,\
    TIMESTAMPDIFF(SECOND, v.time, p.time) AS time_difference FROM small_table v JOIN \
    small_table p ON v.product_id = p.product_id\
          AND v.user_session = p.user_session\
          AND v.event_type = 'view'\
          AND p.event_type = 'purchase'\
          AND v.time < p.time) \
    SELECT brand, AVG(time_difference) as avg_vtp_time, COUNT(*) as num_purchases FROM CTE GROUP BY brand HAVING num_purchases > 100 ORDER BY avg_vtp_time ASC LIMIT 30")


In [140]:
view_to_purchase.show(20)

+--------+------------------+-------------+
|   brand|      avg_vtp_time|num_purchases|
+--------+------------------+-------------+
|   apple|249.96059957173446|        14010|
|  huawei| 317.2927802153969|         2507|
| samsung|  328.220799664183|        19058|
|cordiant| 388.9107573564882|         2073|
|  xiaomi|401.90677091016374|         6779|
+--------+------------------+-------------+



In [9]:
vtp_df = view_to_purchase.toPandas()

23/06/03 19:18:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


In [11]:
vtp_df.to_csv("vtp.csv", index=False)

In [9]:
view_to_cart = spark.sql(
    "WITH CTE AS (SELECT \
    v.brand, \
    v.product_id,\
    v.user_session,\
    v.time AS view_time,\
    p.time AS cart_time,\
    TIMESTAMPDIFF(SECOND, v.time, p.time) AS time_difference FROM small_table v JOIN \
    small_table p ON v.product_id = p.product_id\
          AND v.user_session = p.user_session\
          AND v.event_type = 'view'\
          AND p.event_type = 'cart'\
          AND v.time < p.time) \
    SELECT brand, AVG(time_difference) as avg_vtc_time, COUNT(*) as num_carts FROM CTE GROUP BY brand HAVING num_carts > 100 ORDER BY avg_vtc_time ASC LIMIT 30")


In [10]:
vtc_df = view_to_cart.toPandas()

23/06/03 19:49:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


In [11]:
len(vtc_df)

5

In [12]:
print(vtc_df)

      brand  avg_vtc_time  num_carts
0     apple    215.145040      49552
1  cordiant    224.126629       4375
2   samsung    250.490482      66665
3    huawei    278.584385      10541
4    xiaomi    463.716211      28641


In [13]:
vtp_df = pd.read_csv("vtp.csv")

In [14]:
v_times = vtp_df.merge(vtc_df, on="brand")

In [15]:
v_times

,brand,avg_vtp_time,num_purchases,avg_vtc_time,num_carts
0,apple,243.120771,43032,215.145040,49552
1,samsung,315.013421,52903,250.490482,66665
2,huawei,321.108403,6974,278.584385,10541
3,xiaomi,367.188554,19745,463.716211,28641
4,cordiant,380.542930,9364,224.126629,4375


In [16]:
v_times.to_csv("v_times.csv", index=False)

In [82]:
weekly_purchases = spark.sql(
"SELECT product_id, COUNT(*) as purchase_count,\
    CASE WHEN day BETWEEN 1 AND 7 THEN 1 \
         WHEN day BETWEEN 8 AND 15 THEN 2\
         WHEN day BETWEEN 16 AND 23 THEN 3\
         ELSE 4 END as purchase_week \
    FROM small_table\
    WHERE event_type = 'purchase'\
    GROUP BY product_id, purchase_week\
    ORDER BY product_id, purchase_count DESC")

In [80]:
weekly_purchases.show(20)

+----------+--------------+-------------+
|product_id|purchase_count|purchase_week|
+----------+--------------+-------------+
|   1002524|             4|            1|
|   1002528|             1|            1|
|   1002532|             3|            1|
|   1002536|             1|            1|
|   1002540|             1|            1|
|   1002544|            24|            1|
|   1002547|             2|            1|
|   1002629|             6|            1|
|   1002633|             8|            1|
|   1003304|             4|            1|
|   1003306|             7|            1|
|   1003309|             1|            1|
|   1003316|             2|            1|
|   1003317|             6|            1|
|   1003441|             1|            1|
|   1003533|             1|            1|
|   1003535|             1|            1|
|   1003772|             2|            1|
|   1003894|             1|            1|
|   1003983|             1|            1|
+----------+--------------+-------

### Sankey Test

In [141]:
temp_spark = df[["category_code"]].filter(df.event_type == "purchase")
temp_spark = temp_spark.na.fill("unknown")

In [142]:
temp_spark = temp_spark.withColumn("test", udf(lambda x : x.replace(".", "-") if not None else "empty")("category_code"))
cat_col = split(temp_spark["test"], "-")

temp_spark = temp_spark.withColumn("main_category", cat_col.getItem(0))
temp_spark = temp_spark.withColumn("subcategory_1", cat_col.getItem(1))
temp_spark = temp_spark.withColumn("subcategory_2", cat_col.getItem(2))
# temp_spark = temp_spark.withColumn("subcategory_3", cat_col.getItem(3))

temp_spark = temp_spark.drop("category_code")
temp_spark = temp_spark.drop("test")

In [143]:
grouped_spark = temp_spark.groupby("main_category", "subcategory_1").count().orderBy("count", ascending=False)
grouped_spark[["main_category"]].distinct().count()

14

In [ ]:
top_categories = list(temp_spark.groupby("main_category").count().orderBy("count", ascending=False).limit(15).toPandas()["main_category"])

In [25]:
grouped_spark = temp_spark.filter(temp_spark["main_category"].isin(top_categories)).groupby("main_category", "subcategory_1").count().orderBy("count", ascending=False)
grouped_spark = grouped_spark.na.drop("any")

In [13]:
grouped_spark.show()

+-------------+-------------+-------+
|main_category|subcategory_1|  count|
+-------------+-------------+-------+
|  accessories|     umbrella|   8912|
|  accessories|       wallet| 113986|
|  accessories|          bag| 505378|
|      apparel|       jacket|    437|
|      apparel|       shorts|   1284|
|      apparel|        glove|   3824|
|      apparel|        scarf|   6141|
|      apparel|         sock|   6224|
|      apparel|         belt|   7167|
|      apparel|        skirt|   9866|
|      apparel|       jumper|  34339|
|      apparel|       tshirt|  40522|
|      apparel|     trousers|  41422|
|      apparel|        shirt|  64609|
|      apparel|    underwear|  92183|
|      apparel|        jeans| 109707|
|      apparel|        dress| 133886|
|      apparel|      costume| 380291|
|      apparel|        shoes|3622123|
|   appliances|ironing_board|  99103|
+-------------+-------------+-------+
only showing top 20 rows



In [26]:
grouped_spark_2 = temp_spark.groupby("main_category", "subcategory_1", "subcategory_2").count()
grouped_spark_2 = grouped_spark_2.na.drop("any")

In [365]:
grouped_spark_2.show()

+-------------+-------------+---------------+-----+
|main_category|subcategory_1|  subcategory_2|count|
+-------------+-------------+---------------+-----+
|      apparel|        shoes|           keds|    4|
|      apparel|        shoes|      moccasins|    1|
|   appliances|      kitchen|         washer|   12|
|   appliances|      kitchen|  refrigerators|   22|
|   appliances|      kitchen|     dishwasher|   10|
|   appliances|  environment|         vacuum|    4|
|   appliances|  environment|   water_heater|   11|
|   appliances|     personal|       massager|   19|
|   appliances|      kitchen|           hood|    3|
|   appliances|  environment|air_conditioner|    1|
|   appliances|      kitchen|        blender|    2|
|         auto|  accessories|     compressor|    2|
|         auto|  accessories|         player|   12|
|         auto|  accessories|  videoregister|    1|
|    computers|   components|    motherboard|   13|
|    computers|  peripherals|        printer|    2|
|    compute

In [27]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="main_category", outputCol="main_cat_index") 
indexed_model_1 = indexer.fit(grouped_spark)
indexed_spark = indexed_model_1.transform(grouped_spark) 

indexer = StringIndexer(inputCol="subcategory_1", outputCol="subcat_1_index") 
indexed_model_2 = indexer.fit(indexed_spark)
indexed_spark = indexed_model_2.transform(indexed_spark) 

In [28]:
max_val = indexed_spark.agg({"main_cat_index": "max"}).collect()[0][0]
indexed_spark = indexed_spark.withColumn("subcat_1_index_adjusted", indexed_spark["subcat_1_index"] + max_val + 1)

In [ ]:
indexed_spark_2 = indexed_model_2.transform(grouped_spark_2)
indexer = StringIndexer(inputCol="subcategory_2", outputCol="subcat_2_index") 
indexed_model_3 = indexer.fit(indexed_spark_2)
indexed_spark_2 = indexed_model_3.transform(indexed_spark_2) 

In [354]:
indexed_spark_2 = indexed_spark_2.withColumn("subcat_1_index_adjusted", indexed_spark_2["subcat_1_index"] + max_val + 1)
max_val_2 = indexed_spark_2.agg({"subcat_1_index_adjusted": "max"}).collect()[0][0]
indexed_spark_2 = indexed_spark_2.withColumn("subcat_2_index_adjusted", indexed_spark_2["subcat_2_index"] + max_val_2 + 1)

In [29]:
indexed_spark.show(1000)

+-------------+--------------+------+--------------+--------------+-----------------------+
|main_category| subcategory_1| count|main_cat_index|subcat_1_index|subcat_1_index_adjusted|
+-------------+--------------+------+--------------+--------------+-----------------------+
|  electronics|    smartphone|720665|           1.0|          42.0|                   55.0|
|   appliances|       kitchen|109465|           2.0|           1.0|                   14.0|
|  electronics|         audio| 81894|           1.0|           4.0|                   17.0|
|  electronics|         video| 52020|           1.0|          56.0|                   69.0|
|   appliances|   environment| 44049|           2.0|          20.0|                   33.0|
|  electronics|        clocks| 41143|           1.0|          13.0|                   26.0|
|    computers|      notebook| 34023|           4.0|          31.0|                   44.0|
|         auto|   accessories| 21339|          10.0|           3.0|             

In [ ]:
indexed_spark_2.show(1000)

In [30]:
sankey_df = indexed_spark.limit(1000).toPandas()

In [327]:
sankey_df_2 = indexed_spark_2.limit(1000).toPandas()

In [38]:
labels = indexed_model_1.labels + indexed_model_2.labels

In [352]:
labels

['kitchen',
 'accessories',
 'audio',
 'bag',
 'bathroom',
 'bedroom',
 'bicycle',
 'camera',
 'carriage',
 'clocks',
 'components',
 'costume',
 'desktop',
 'dolls',
 'environment',
 'jeans',
 'living_room',
 'notebook',
 'peripherals',
 'personal',
 'sewing_machine',
 'shirt',
 'shoes',
 'smartphone',
 'tablet',
 'telephone',
 'tools',
 'toys',
 'trousers',
 'tshirt',
 'underwear',
 'video',
 'air_conditioner',
 'bath',
 'bed',
 'blender',
 'cabinet',
 'chair',
 'compressor',
 'cpu',
 'dishwasher',
 'drill',
 'hdd',
 'headphone',
 'hood',
 'keds',
 'keyboard',
 'light',
 'massager',
 'memory',
 'moccasins',
 'motherboard',
 'music_tools',
 'painting',
 'player',
 'printer',
 'refrigerators',
 'saw',
 'sofa',
 'subwoofer',
 'table',
 'tv',
 'vacuum',
 'video',
 'videocards',
 'videoregister',
 'washer',
 'water_heater',
 'welding']

In [196]:
print(sankey_df["main_cat_index"])

0    1.0
1    2.0
2    2.0
3    3.0
4    4.0
5    4.0
6    1.0
7    2.0
8    1.0
9    1.0
Name: main_cat_index, dtype: float64


In [34]:
import plotly.graph_objects as go

source = sankey_df["main_cat_index"] 
target = sankey_df["subcat_1_index_adjusted"]
value = sankey_df["count"]
# data to dict, dict to sankey
link = dict(source = source, target = target, value = value)
node = dict(label = labels, pad=50, thickness=5)
data = go.Sankey(link = link, node = node)
# plot
fig = go.Figure(data)
fig.update_layout(
    width=1000,
    height=1000
)

fig.show()

In [36]:
sankey_df.to_csv("sankey.csv", index=False)

In [40]:
import pickle

with open("sankey_labels.txt", "wb") as f:
    pickle.dump(labels, f)

# Messages Data

In [8]:
import pickle
import pandas as pd
from pyspark.sql.functions import concat_ws, col, udf
from pyspark.sql.types import StringType, DateType
from pandasql import sqldf
from pyspark.sql import functions as f
from datetime import datetime

In [80]:
campaigns_df = pd.read_csv("data/campaigns.csv")

In [16]:
campaigns_df.columns

Index(['id', 'campaign_type', 'channel', 'topic', 'started_at', 'finished_at',
       'total_count', 'ab_test', 'warmup_mode', 'hour_limit', 'subject_length',
       'subject_with_personalization', 'subject_with_deadline',
       'subject_with_emoji', 'subject_with_bonuses', 'subject_with_discount',
       'subject_with_saleout', 'is_test', 'position'],
      dtype='object')

In [ ]:
holidays_df

In [90]:
holidays_df = pd.read_csv("data/holidays.csv")
holidays_df["date"] = pd.to_datetime(holidays_df["date"]).dt.date

In [86]:
campaigns_df["started_at"] = pd.to_datetime(campaigns_df["started_at"]).dt.date
# campaigns_df["month"] = campaigns_df["started_at"].apply(lambda x : int(x.month))
# campaigns_df["year"] = campaigns_df["started_at"].apply(lambda x : int(x.year))

In [502]:
other_msgs_df = spark.sql("SELECT sent_at, message_type, COUNT(*) as count FROM temp_table WHERE message_type = 'transactional' or message_type = 'trigger' GROUP BY sent_at, message_type").toPandas()

In [506]:
with open("other_msgs.txt", "wb") as f:
    pickle.dump(other_msgs_df, f)

In [505]:
fig = px.area(other_msgs_df, x="sent_at", y="count", color="message_type")
fig

In [122]:
holidays = []
for date in list(temp_df["started_at"]):
    if date in list(holidays_df["date"]):
        holidays.append(holidays_df[holidays_df["date"] == date]["holiday"])
    else:
        holidays.append("")

In [125]:
temp_temp_df = temp_df.copy()
temp_temp_df["count"] = 0

In [136]:
import plotly.graph_objects as go

In [138]:
fig = px.area(temp_df, x="started_at", y="count", color="channel")
fig 

In [10]:
subject_cols = ["subject_with_emoji", "subject_with_personalization", "subject_with_deadline", "subject_with_bonuses", "subject_with_discount", "subject_with_saleout"]

In [48]:
func =  udf (lambda x: datetime.strptime(f"{x[5:7]}/{x[8:10]}/{x[:4]}" , '%m/%d/%Y'), DateType())

def setup_data():
    campaigns_df = spark.read.option("header",True).csv("data/campaigns.csv")
    messages_df = spark.read.option("header",True).csv("data/messages-demo.csv")
    for col_ in subject_cols:
        campaigns_df = campaigns_df.withColumn(col_, campaigns_df[col_].cast("boolean"))

    campaigns_df = campaigns_df.withColumn("campaign_uid", concat_ws("_", campaigns_df["campaign_type"], campaigns_df["id"]))
    campaigns_df = campaigns_df.withColumnRenamed("channel", "c_channel")
    campaigns_df = campaigns_df.withColumnRenamed("id", "c_id")
    messages_df = messages_df.withColumn("campaign_uid", concat_ws("_", messages_df["message_type"], messages_df["campaign_id"]))
    merged_df = messages_df.join(campaigns_df, ["campaign_uid"])
    # merged_df = merged_df.na.drop(subset=["started_at"])
    # merged_df = merged_df.withColumn("started_at", func(col('started_at')))
    merged_df = merged_df.na.drop(subset=["sent_at"])
    merged_df = merged_df.withColumn("sent_at", func(col('sent_at')))

    merged_df = merged_df.withColumn("purchased_at", f.to_date(merged_df["purchased_at"]))
    merged_df = merged_df.withColumn("clicked_first_time_at", f.to_date(merged_df["clicked_first_time_at"]))

    merged_df = merged_df.withColumn("time_to_purchase", merged_df["purchased_at"] - merged_df["clicked_first_time_at"])
    merged_df = merged_df.withColumn("time_to_purchase", f.udf(lambda x : x.total_seconds()/60 if x != None else 0)("time_to_purchase"))
    print(merged_df.columns)
    return merged_df

merged_df = setup_data()

['campaign_uid', 'id', 'message_id', 'campaign_id', 'message_type', 'client_id', 'channel', 'category', 'platform', 'email_provider', 'stream', 'date', 'sent_at', 'is_opened', 'opened_first_time_at', 'opened_last_time_at', 'is_clicked', 'clicked_first_time_at', 'clicked_last_time_at', 'is_unsubscribed', 'unsubscribed_at', 'is_hard_bounced', 'hard_bounced_at', 'is_soft_bounced', 'soft_bounced_at', 'is_complained', 'complained_at', 'is_blocked', 'blocked_at', 'is_purchased', 'purchased_at', 'created_at', 'updated_at', 'c_id', 'campaign_type', 'c_channel', 'topic', 'started_at', 'finished_at', 'total_count', 'ab_test', 'warmup_mode', 'hour_limit', 'subject_length', 'subject_with_personalization', 'subject_with_deadline', 'subject_with_emoji', 'subject_with_bonuses', 'subject_with_discount', 'subject_with_saleout', 'is_test', 'position', 'time_to_purchase']


In [49]:
merged_df.count()

10000000

In [34]:
merged_df.show(5)

+------------+---+----------+-----------+------------+---------+-------+--------+--------+--------------+------+----+-------+---------+--------------------+-------------------+----------+---------------------+--------------------+---------------+---------------+---------------+---------------+---------------+---------------+-------------+-------------+----------+----------+------------+------------+----------+----------+----+-------------+---------+-----+----------+-----------+-----------+-------+-----------+----------+--------------+----------------------------+---------------------+------------------+--------------------+---------------------+--------------------+-------+--------+----------------+
|campaign_uid| id|message_id|campaign_id|message_type|client_id|channel|category|platform|email_provider|stream|date|sent_at|is_opened|opened_first_time_at|opened_last_time_at|is_clicked|clicked_first_time_at|clicked_last_time_at|is_unsubscribed|unsubscribed_at|is_hard_bounced|hard_bounced_

## Set up pandas df

In [12]:
temp_df = merged_df.limit(200000).toPandas()

23/07/11 22:33:21 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [52]:
temp_table = merged_df.createOrReplaceTempView("temp_table")

In [267]:
print(type(temp_table))

<class 'NoneType'>


In [180]:
temp_df.columns

Index(['campaign_uid', 'id', 'message_id', 'campaign_id', 'message_type',
       'client_id', 'channel', 'category', 'platform', 'email_provider',
       'stream', 'date', 'sent_at', 'is_opened', 'opened_first_time_at',
       'opened_last_time_at', 'is_clicked', 'clicked_first_time_at',
       'clicked_last_time_at', 'is_unsubscribed', 'unsubscribed_at',
       'is_hard_bounced', 'hard_bounced_at', 'is_soft_bounced',
       'soft_bounced_at', 'is_complained', 'complained_at', 'is_blocked',
       'blocked_at', 'is_purchased', 'purchased_at', 'created_at',
       'updated_at', 'c_id', 'campaign_type', 'c_channel', 'topic',
       'started_at', 'finished_at', 'total_count', 'ab_test', 'warmup_mode',
       'hour_limit', 'subject_length', 'subject_with_personalization',
       'subject_with_deadline', 'subject_with_emoji', 'subject_with_bonuses',
       'subject_with_discount', 'subject_with_saleout', 'is_test', 'position'],
      dtype='object')

In [172]:
temp_df["is_opened"] = temp_df["is_opened"].fillna("f")

In [204]:
temp_df.columns

Index(['id', 'message_id', 'campaign_id', 'message_type', 'client_id',
       'channel', 'category', 'platform', 'email_provider', 'stream', 'date',
       'sent_at', 'is_opened', 'opened_first_time_at', 'opened_last_time_at',
       'is_clicked', 'clicked_first_time_at', 'clicked_last_time_at',
       'is_unsubscribed', 'unsubscribed_at', 'is_hard_bounced',
       'hard_bounced_at', 'is_soft_bounced', 'soft_bounced_at',
       'is_complained', 'complained_at', 'is_blocked', 'blocked_at',
       'is_purchased', 'purchased_at', 'created_at', 'updated_at',
       'campaign_uid', 'c_id', 'campaign_type', 'c_channel', 'topic',
       'started_at', 'finished_at', 'total_count', 'ab_test', 'warmup_mode',
       'hour_limit', 'subject_length', 'subject_with_personalization',
       'subject_with_deadline', 'subject_with_emoji', 'subject_with_bonuses',
       'subject_with_discount', 'subject_with_saleout', 'is_test', 'position',
       'campaign_uid'],
      dtype='object')

## Important column unique values

In [154]:
temp_df["topic"].unique()

array(['order ready for pickup', 'order pickup still pending',
       'profile updated', 'order shipped', 'order created',
       'order cancelled', 'abandoned view', 'abandoned cart',
       'abandoned category', 'subscribed', 'sale out', 'recent purchase',
       'added to wish list', 'abandoned search'], dtype=object)

In [ ]:
temp_df.head(100)

In [143]:
for col in subject_cols:
    temp_df[col] = temp_df[col].astype(bool)

In [140]:
temp_df[temp_df["is_purchased"] == 't'].groupby("message_type").count()["id"]

message_type
transactional     95
trigger          140
Name: id, dtype: int64

In [227]:
temp_df[(temp_df["is_purchased"] == 't') & (temp_df["message_type"] == 'trigger')][['topic', 'clicked_first_time_at', 'opened_first_time_at', "purchased_at","time_to_purchase"]].head(50)

,topic,clicked_first_time_at,opened_first_time_at,purchased_at,time_to_purchase
13208,abandoned category,2021-05-04 06:40:50,2021-04-30 10:44:06,2021-04-30 10:41:55,-5518.916667
13323,abandoned view,2021-04-30 15:33:10,2021-04-30 15:32:34,2021-04-30 15:32:02,-1.133333
13471,abandoned category,2021-05-04 05:41:36,2021-05-04 05:41:11,2021-04-30 17:15:19,-5066.283333
13810,abandoned category,2021-04-30 15:00:46,2021-04-30 15:00:36,2021-04-30 12:02:29,-178.283333
13891,abandoned cart,2021-04-30 21:47:16,2021-04-30 21:46:29,2021-04-30 19:00:37,-166.650000
14145,abandoned view,2021-04-30 21:19:35,2021-04-30 21:19:04,2021-04-30 12:06:21,-553.233333
14186,abandoned category,2021-04-30 10:59:29,2021-04-30 10:59:16,2021-04-30 10:15:24,-44.083333
14225,abandoned view,2021-04-30 15:03:10,2021-04-30 15:02:37,2021-04-30 11:31:54,-211.266667
14261,abandoned cart,2021-04-30 08:50:27,2021-04-30 08:50:09,2021-04-30 05:35:29,-194.966667
14607,added to wish list,2021-04-30 19:23:07,2021-04-30 19:22:59,2021-04-30 10:27:01,-536.100000


## Conversion rate

In [155]:
q = "SELECT message_type, channel, is_opened_count * 1.0 / total * 100 as open_rate FROM \
    (SELECT message_type, channel, COUNT(*) as total, SUM(CASE WHEN is_opened = 't' THEN 1 ELSE 0 END) as is_opened_count FROM temp_df GROUP BY message_type, channel)"
chart_1_df = sqldf(q, globals())

In [156]:
chart_1_df

,message_type,channel,open_rate
0,bulk,mobile_push,33.746519
1,transactional,email,35.574572
2,transactional,mobile_push,0.000000
3,trigger,email,17.417359
4,trigger,web_push,0.000000


In [288]:
chart_2_df = spark.sql("SELECT message_type, channel, topic, total, is_opened_count * 1.0 / total * 100 as open_rate FROM \
    (SELECT message_type, channel, topic, COUNT(*) as total, SUM(CASE WHEN is_opened = 't' THEN 1 ELSE 0 END) as is_opened_count FROM temp_table GROUP BY message_type, channel, topic)").toPandas()
# q = "SELECT message_type, channel, topic, total, is_opened_count * 1.0 / total * 100 as open_rate FROM \
#     (SELECT message_type, channel, topic, COUNT(*) as total, SUM(CASE WHEN is_opened = 't' THEN 1 ELSE 0 END) as is_opened_count FROM temp_df GROUP BY message_type, channel, topic) \
#     ORDER BY open_rate"
# chart_2_df = sqldf(q, globals())

In [290]:
chart_2_df = chart_2_df[chart_2_df["open_rate"] > 0]
chart_2_df["topic"] = chart_2_df["topic"].fillna("unknown")
chart_2_df

,message_type,channel,topic,total,open_rate
0,trigger,web_push,abandoned cart,46,8.695652173913
1,transactional,email,order created,201817,37.843194577266
3,trigger,email,subscribed,302377,26.412392476941
4,transactional,email,order cancelled,142985,30.243032485925
5,trigger,email,added to wish list,83078,17.859120344736
6,bulk,mobile_push,sale out,4152975,11.762724312090
7,trigger,email,recent purchase,472521,18.015918869214
8,trigger,email,abandoned cart,241364,13.432823453373
10,trigger,email,abandoned view,557393,16.686072483867
11,transactional,email,order shipped,36285,51.048642689817


In [426]:
with open("open_rate_bars.txt", "wb") as f:
    pickle.dump(chart_2_df, f)

In [292]:
px.sunburst(chart_2_df, path=["message_type", "channel", "topic"], color="message_type", values="total")

In [302]:
# discrete_map = {"trigger":"#e377c2", "email":"#17becf", "bulk":"rgb(255,0,0)"}
fig = px.bar(chart_2_df, x="topic", y="open_rate", color="message_type")
fig.update_layout(xaxis={"categoryorder":"total descending"})


### Subject effect on conversion

In [247]:
for col in subject_cols:
    print(col)
    print(merged_df.filter(merged_df[col] == True).count())

subject_with_emoji


7231395
subject_with_personalization


6373
subject_with_deadline


273976
subject_with_bonuses


168998
subject_with_discount


30798
subject_with_saleout


231972


In [182]:
q = "SELECT campaign_type, channel, \
    SUM(CASE WHEN subject_with_discount = True and is_opened = True THEN 1 ELSE 0 END)/SUM(CASE WHEN subject_with_discount = True and is_opened = 'f' THEN 1 ELSE 0 END) * 1.0 / 100 as percent_discount \
    FROM temp_df GROUP BY campaign_type, channel"
sqldf(q, globals())

,campaign_type,channel,percent_discount
0,bulk,mobile_push,None
1,transactional,email,None
2,transactional,mobile_push,None
3,trigger,email,None
4,trigger,web_push,None


In [404]:
pers_df = spark.sql("SELECT campaign_type, \
    SUM(CASE WHEN subject_with_personalization = True and is_opened = 't' THEN 1 ELSE 0 END) as a, \
    SUM(CASE WHEN subject_with_personalization = True and is_opened = 'f' THEN 1 ELSE 0 END) as b \
    FROM temp_table GROUP BY campaign_type").toPandas()
saleout_df = spark.sql("SELECT campaign_type, \
    SUM(CASE WHEN subject_with_saleout = True and is_opened = 't' THEN 1 ELSE 0 END) as a, \
    SUM(CASE WHEN subject_with_saleout = True and is_opened = 'f' THEN 1 ELSE 0 END) as b \
    FROM temp_table GROUP BY campaign_type").toPandas()
discount_df = spark.sql("SELECT campaign_type, \
    SUM(CASE WHEN subject_with_discount = True and is_opened = 't' THEN 1 ELSE 0 END) as a, \
    SUM(CASE WHEN subject_with_discount = True and is_opened = 'f' THEN 1 ELSE 0 END) as b \
    FROM temp_table GROUP BY campaign_type").toPandas()
deadline_df = spark.sql("SELECT campaign_type, \
    SUM(CASE WHEN subject_with_deadline = True and is_opened = 't' THEN 1 ELSE 0 END) as a, \
    SUM(CASE WHEN subject_with_deadline = True and is_opened = 'f' THEN 1 ELSE 0 END) as b \
    FROM temp_table GROUP BY campaign_type").toPandas()
# q = "SELECT campaign_type, \
#     SUM(CASE WHEN subject_with_discount = True and is_opened = 't' THEN 1 ELSE 0 END) as a, \
#     SUM(CASE WHEN subject_with_discount = True and is_opened = 'f' THEN 1 ELSE 0 END) as b \
#     FROM temp_df GROUP BY campaign_type"

# chart_df_3 = sqldf(q, globals())

In [427]:
pers_df = spark.sql("SELECT campaign_type, \
    SUM(CASE WHEN subject_with_personalization = False and is_opened = 't' THEN 1 ELSE 0 END) as a, \
    SUM(CASE WHEN subject_with_personalization = False and is_opened = 'f' THEN 1 ELSE 0 END) as b \
    FROM temp_table GROUP BY campaign_type").toPandas()
saleout_df = spark.sql("SELECT campaign_type, \
    SUM(CASE WHEN subject_with_saleout = False and is_opened = 't' THEN 1 ELSE 0 END) as a, \
    SUM(CASE WHEN subject_with_saleout = False and is_opened = 'f' THEN 1 ELSE 0 END) as b \
    FROM temp_table GROUP BY campaign_type").toPandas()
discount_df = spark.sql("SELECT campaign_type, \
    SUM(CASE WHEN subject_with_discount = False and is_opened = 't' THEN 1 ELSE 0 END) as a, \
    SUM(CASE WHEN subject_with_discount = False and is_opened = 'f' THEN 1 ELSE 0 END) as b \
    FROM temp_table GROUP BY campaign_type").toPandas()
deadline_df = spark.sql("SELECT campaign_type, \
    SUM(CASE WHEN subject_with_deadline = False and is_opened = 't' THEN 1 ELSE 0 END) as a, \
    SUM(CASE WHEN subject_with_deadline = False and is_opened = 'f' THEN 1 ELSE 0 END) as b \
    FROM temp_table GROUP BY campaign_type").toPandas()

In [428]:
pers_reshape = pd.DataFrame({"Type": ["Opened", "Not Opened"], "Count": [pers_df[pers_df["campaign_type"] == "bulk"]["a"][0],
                                                                         pers_df[pers_df["campaign_type"] == "bulk"]["b"][0]]})
discount_reshape = pd.DataFrame({"Type": ["Opened", "Not Opened"], "Count": [discount_df[discount_df["campaign_type"] == "bulk"]["a"][0],
                                                                         discount_df[discount_df["campaign_type"] == "bulk"]["b"][0]]})
saleout_reshape = pd.DataFrame({"Type": ["Opened", "Not Opened"], "Count": [saleout_df[saleout_df["campaign_type"] == "bulk"]["a"][0],
                                                                         saleout_df[saleout_df["campaign_type"] == "bulk"]["b"][0]]})
deadline_reshape = pd.DataFrame({"Type": ["Opened", "Not Opened"], "Count": [deadline_df[deadline_df["campaign_type"] == "bulk"]["a"][0],
                                                                         deadline_df[deadline_df["campaign_type"] == "bulk"]["b"][0]]})

final_dfs = [pers_reshape, discount_reshape, saleout_reshape, deadline_reshape]

In [432]:
with open("subject_pies_missing.txt", "wb") as f:
    pickle.dump(final_dfs, f)

In [429]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}], [{"type": "pie"}, {"type": "pie"}]], subplot_titles=("With Emoji", "With Discount", "With Saleout", "With Deadline"))


In [430]:
col = 1
row = 1
for i in range(4):
    fig.add_trace(go.Pie(
        values=final_dfs[i]["Count"],
        labels=final_dfs[i]["Type"], hole=0.5, 
        domain=dict(x=[0, 0.5])), 
        row=row, col=col)
    col += 1
    
    if col == 3:
        col = 1
        row += 1  


fig.update_layout(height=600, width=800)
# px.pie(chart_df_3_reshape, names="Type", values="Count", title="Bulk Message Open Rate") 

subject_length - the lenght of subject in symbols (sometimes open rate depends on subject length)
subject_with_personalization - subject includes personalization. Example: "Hello, John Wick, there is the special delivery for you!"
subject_with_deadline - subject includes some kind of deadline. Example: "Hey, John Wick! Only 4 days left when you can get your special delivery!"
subject_with_emoji - subject includes emoji. Example: "Hello, John Wick, there is the special delivery for you 😵🍺"
subject_with_bonuses - subject includes information about bonuses. Example: "Hey, John Wick! We added 300 gold coins to your bonus account!"
subject_with_discount - subject includes information about discount. Example: "Hey, John Wick! 40% discount in Continental for a low season!"
subject_with_saleout - subject announces some kind of saleout. Example: "Hey, John Wick! Black Friday is coming!"

## Purchase rate

In [13]:
temp_df["clicked_first_time_at"] = pd.to_datetime(temp_df["clicked_first_time_at"])
temp_df["purchased_at"] = pd.to_datetime(temp_df["purchased_at"])

In [14]:
temp_df["time_to_purchase"] = temp_df["purchased_at"] - temp_df["clicked_first_time_at"]
temp_df["time_to_purchase"] = temp_df["time_to_purchase"].apply(lambda x : x.total_seconds()/60)

In [15]:
temp_df["time_to_purchase"]

0         3532.866667
1                 NaN
2                 NaN
3                 NaN
4                 NaN
             ...     
199995            NaN
199996            NaN
199997            NaN
199998            NaN
199999            NaN
Name: time_to_purchase, Length: 200000, dtype: float64

In [53]:
purch_df = spark.sql("SELECT message_type, num_purchases/(total * 1.0) * 100 as percent_purchases FROM \
    (SELECT \
        message_type, \
        SUM(CASE WHEN is_purchased = 't' THEN 1 END) as num_purchases, \
        COUNT(*) as total\
    FROM temp_table \
    GROUP BY message_type)").toPandas()


# q = "SELECT message_type, num_purchases/(total * 1.0) * 100 as percent_purchases FROM \
#     (SELECT \
#         message_type, \
#         SUM(CASE WHEN is_purchased = 't' THEN 1 END) as num_purchases, \
#         COUNT(*) as total\
#     FROM temp_df \
#     GROUP BY message_type)"
# sqldf(q, globals())

In [54]:
purch_df.head()

,message_type,percent_purchases
0,bulk,0.0353421183607
1,trigger,0.2184206563936
2,transactional,0.6473767662698


In [59]:
atp_df = spark.sql("SELECT \
        message_type, \
        ABS(AVG(time_to_purchase)) as avg_time_to_purchase\
    FROM temp_table \
    WHERE is_purchased = 't'\
    GROUP BY message_type").toPandas()

# q = "SELECT \
#         message_type, \
#         AVG(time_to_purchase) as avg_time_to_purchase\
#     FROM temp_df \
#     WHERE is_purchased = 't'\
#     GROUP BY message_type"
# sqldf(q, globals())

In [60]:
atp_df.head()

,message_type,avg_time_to_purchase
0,trigger,910.674374
1,transactional,1595.886333
2,bulk,1891.730769


In [62]:
purch_atp_df = pd.merge(purch_df, atp_df, on="message_type")

In [65]:
with open("purch_atp.txt", "wb") as f:
    pickle.dump(purch_atp_df, f)


In [67]:
fig = px.line(purch_atp_df, x="avg_time_to_purchase", y="percent_purchases", color="message_type")
fig

In [84]:
unsub_df = spark.sql("SELECT message_type, unsubscribed/total * 100 as percent_unsubscribed FROM \
    (SELECT \
        message_type, \
        SUM(CASE WHEN is_unsubscribed = 't' THEN 1 END) as unsubscribed, \
        COUNT(*) as total\
    FROM temp_table \
    GROUP BY message_type)").toPandas()


# q = "SELECT message_type, unsubscribed/(total * 1.0) * 100 as percent_unsubscribed FROM \
#     (SELECT \
#         message_type, \
#         SUM(CASE WHEN is_unsubscribed = 't' THEN 1 END) as unsubscribed, \
#         COUNT(*) as total\
#     FROM temp_df \
#     GROUP BY message_type)"
# sqldf(q, globals())

In [85]:
unsub_df

,message_type,percent_unsubscribed
0,bulk,4.145676
1,trigger,0.448204
2,transactional,0.101368


In [89]:
blocked_df = spark.sql("SELECT message_type, blocked/(total * 1.0) * 100 as percent_blocked FROM \
    (SELECT \
        message_type, \
        SUM(CASE WHEN is_blocked = 't' THEN 1 ELSE 0 END) as blocked, \
        COUNT(*) as total\
    FROM temp_table \
    GROUP BY message_type)").toPandas()

# q = "SELECT message_type, blocked/(total * 1.0) * 100 as percent_blocked FROM \
#     (SELECT \
#         message_type, \
#         SUM(CASE WHEN is_blocked = 't' THEN 1 ELSE 0 END) as blocked, \
#         COUNT(*) as total\
#     FROM temp_df \
#     GROUP BY message_type)"
# sqldf(q, globals())

In [90]:
blocked_df

,message_type,percent_blocked
0,bulk,0.0006230181121
1,trigger,0.0081831759514
2,transactional,0E-13


In [91]:
unsub_blocked_df = pd.merge(unsub_df, blocked_df, on="message_type")

In [92]:
with open("unsub_blocked.txt", "wb") as f:
    pickle.dump(unsub_blocked_df, f)
